In [ ]:
from tensorflow import keras

lstm_torque = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_torque_new.h5')
lstm_d50 = keras.models.load_model('Bidirectional_LSTM_Multivariate_alldatafiles_d50.h5')

In [ ]:
from HelperFunctions.FileReader import FileReader
import pandas as pd
# Reading data files 
run_name_all = ['Run1.xlsx','Run2.xlsx','Run5.xlsx','Run3.xlsx','Run4.xlsx','Run6.xlsx']
# run_name_all = ['Run2.xlsx']
run_time = 29 # minutes
run_path_tsg = './TSGscreen/'
run_path_feeder = './Feederdata/'
run_path_eyecon = './Eyecondata/'
combined_datafile = pd.DataFrame()
for run_name in run_name_all:

    fr_obj = FileReader(run_name)

    tsg_data = fr_obj.read_excelFile(run_path_tsg,0)
    feeder_data = fr_obj.read_excelFile(run_path_feeder,1)
    eyecon_data = fr_obj.read_excelFile(run_path_eyecon,0)

    print(len(tsg_data))
    print(len(eyecon_data))
    print(len(feeder_data))

    combined_datafile = combined_datafile.append(fr_obj.combine_datafiles(tsg_data,eyecon_data,feeder_data))
    print(len(combined_datafile))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler()
Y_scaler_tor = MinMaxScaler()
Y_scaler_d50 = MinMaxScaler()


X_data = X_scaler.fit_transform(combined_datafile[['Mass flow rate','7 NetWeight','Liquid flow rate','Actual RPM','Torque',' D_v50']])
Y_data_Torque = Y_scaler_tor.fit_transform(combined_datafile[['Torque']])
Y_data_d50 = Y_scaler_d50.fit_transform(combined_datafile[[' D_v50']])




In [ ]:
import numpy as np
energy_used = np.trapz(np.multiply(combined_datafile['Actual RPM'],combined_datafile['Torque']) * (2*np.pi/60))
energy_used_per_weight = energy_used / (np.mean(combined_datafile['Mass flow rate']) * (run_time * 60) /3600)
print((energy_used))


In [ ]:
opt_data = pd.read_excel('./optimization_Data.xlsx')
opt_data = opt_data.dropna(axis=1)
# print(opt_data.describe)

In [ ]:
energy_used_opt_30min = np.multiply(opt_data['RPM'],opt_data['Torque']) * (2*np.pi) * 30
energy_used_opt_30min_per_weight = np.divide(energy_used_opt_30min,opt_data['FlowRate (kg/hr)']) * (3600/(30*60))
energy_used_opt_10min = np.multiply(opt_data['RPM'],opt_data['Torque']) * (2*np.pi) * 10
energy_used_opt_10min_per_weight = np.divide(energy_used_opt_30min,opt_data['FlowRate (kg/hr)']) * (3600/(10*60))
opt_data['10 min energy'] = energy_used_opt_10min_per_weight
print(energy_used_opt_30min)
print(energy_used_opt_10min)
# 

In [ ]:
run_name_all = ['Run3.xlsx']
run_time = 29 # minutes
run_feedrate = 1.75
run_rpm = 700
run_ls = 1
run_path_eyecon = './Eyecondata/'
combined_datafile_single = pd.DataFrame()
for run_name in run_name_all:

    fr_obj = FileReader(run_name)

    tsg_data = fr_obj.read_excelFile(run_path_tsg,0)
    feeder_data = fr_obj.read_excelFile(run_path_feeder,1)
    eyecon_data = fr_obj.read_excelFile(run_path_eyecon,0)


    combined_datafile_single = combined_datafile_single.append(fr_obj.combine_datafiles(tsg_data,eyecon_data,feeder_data))
    print(len(combined_datafile_single))

lstm_input_predict = X_scaler.fit_transform(combined_datafile_single[['Mass flow rate','7 NetWeight','Liquid flow rate','Actual RPM','Torque',' D_v50']])
energy_used = np.trapz(np.multiply(combined_datafile_single['Actual RPM'],combined_datafile_single['Torque']) * (2*np.pi/60))
energy_used_per_weight = energy_used / (np.mean(combined_datafile_single['Mass flow rate']) * (run_time * 60) /3600)
print((energy_used))

In [ ]:
lstm_input_predict_200 = lstm_input_predict[:200]
lstm_input_predict_200 = lstm_input_predict_200.reshape(1,200,6)

In [ ]:
torque_predict = lstm_torque.predict(lstm_input_predict_200)
torque_pred_10 = Y_scaler_tor.inverse_transform(torque_predict)
d50_predict = lstm_d50.predict(lstm_input_predict_200)
d50_pred_10 = Y_scaler_d50.inverse_transform(d50_predict)

In [ ]:
print(len(torque_predict[0]))

In [ ]:
energy_data_10mins = np.trapz(np.multiply(combined_datafile_single['Actual RPM'][:120],combined_datafile_single['Torque'][:120]) * (2*np.pi/60))
energy_data_full200 = np.trapz(np.multiply(combined_datafile_single['Actual RPM'],combined_datafile_single['Torque']) * (2*np.pi/60))
print(energy_data_10mins)
print(energy_data_full200)



In [ ]:
d50_dry  = np.array(Y_scaler_d50.inverse_transform(d50_predict)) / 2.1
# print(d50_dry)
# print(combined_datafile_single[' D_v50'][len(lstm_input_predict)-200:len(lstm_input_predict)] / 2.2)

In [ ]:
# print(Y_scaler_tor.inverse_transform(torque_predict))
# print(combined_datafile_single['Torque'][len(lstm_input_predict)-200:len(lstm_input_predict)])

In [ ]:
#keeping RPM constant
min_ss_data = np.argmin(opt_data[(opt_data['RPM']==run_rpm) & (opt_data['final d50']>450)]['10 min energy'])
print(min_ss_data)
print(opt_data['FlowRate (kg/hr)'][min_ss_data])
print(opt_data['RPM'][min_ss_data])
print(opt_data['L/S Ratio'][min_ss_data])
print(opt_data['final d50'][min_ss_data])
combined_datafile_single['Mass flow rate'][120:len(combined_datafile_single)] = np.linspace(opt_data['FlowRate (kg/hr)'][min_ss_data],opt_data['FlowRate (kg/hr)'][min_ss_data],len(combined_datafile_single)-120)
combined_datafile_single['Liquid flow rate'][120:len(combined_datafile_single)] = np.linspace(opt_data['FlowRate (kg/hr)'][min_ss_data],opt_data['FlowRate (kg/hr)'][min_ss_data],len(combined_datafile_single)-120) * opt_data['L/S Ratio'][min_ss_data]
combined_datafile_single[' D_v50'][120:len(combined_datafile_single)] = np.linspace(opt_data['final d50'][min_ss_data],opt_data['final d50'][min_ss_data],len(combined_datafile_single)-120)
# print(combined_datafile_single[' D_v50'][120:320])
print(combined_datafile_single)
lstm_input_predict_10_20 = X_scaler.fit_transform(combined_datafile_single[['Mass flow rate','7 NetWeight','Liquid flow rate','Actual RPM','Torque',' D_v50']])
lstm_input_predict_10_200 = lstm_input_predict_10_20[len(combined_datafile_single)-200:len(combined_datafile_single)]
lstm_input_predict_10_200 = lstm_input_predict_10_200.reshape(1,200,6)
torque_predict_10_20 = lstm_torque.predict(lstm_input_predict_10_200)
torque_predict_10_20 = Y_scaler_tor.inverse_transform(torque_predict_10_20)
d50_predict_10_20 = lstm_d50.predict(lstm_input_predict_10_200)
d50_predict_10_20 = Y_scaler_d50.inverse_transform(d50_predict_10_20)

plt.figure()
plt.plot(combined_datafile_single['Torque'])
plt.plot(np.concatenate([np.reshape(torque_predict_10_20,(200,)),np.reshape(torque_predict_10_20,(200,))[:160]]))
plt.xlabel('Time points')
plt.ylabel('Torque (N/m.s)')
plt.legend(['Experimental','LSTM model prediction'])
plt.rcParams.update({'font.size': 12})
plt.rcParams["figure.figsize"] = (9,6)


In [ ]:
import matplotlib.pyplot as plt
print(d50_pred_10.dtype)
time = np.linspace(0,600,120)
plt.plot(time,np.reshape(d50_pred_10[0,:120],(120,)))
time1 = np.linspace(600,1200,120)
plt.plot(time1,np.reshape(d50_predict_10_20[0,:120],(120,)))
time2 = np.linspace(1200,1740,108)
plt.plot(time2,np.reshape(d50_predict_10_20[0,:120],(120,))[:108])
plt.ylim([800,1000])

plt.plot(time,np.reshape(d50_pred_10[0,:120],(120,))/2.1)
plt.plot(time1,np.reshape(d50_predict_10_20[0,:120],(120,))/2.1)
plt.plot(time2,np.reshape(d50_predict_10_20[0,:120],(120,))[:108]/2.1)
plt.ylim([400,1200])
plt.legend(['D_50 exp Wet','Predicted D_50 10-20 mins Wet','Predicted D_50 20-29 mins Wet','D_50 exp Dry','Predicted D_50 10-20 mins Dry','Predicted D_50 20-29 mins Dry'])
plt.ylabel('D_50 ($\mu$m)')
plt.xlabel('Time (seconds)')
plt.figure()
plt.plot(time,np.reshape(torque_pred_10[0,:120],(120,)))
plt.plot(time1,np.reshape(torque_predict_10_20[0,:120],(120,)))
plt.plot(time2,np.reshape(torque_predict_10_20[0,:120],(120,))[:108])
plt.legend(['Torque exp','Predicted Torque 10-20 mins','Predicted Torque 20-29 mins'])
plt.xlabel('Time (seconds)')
plt.ylabel('Torque (N/m.s)')
plt.ylim([1,4])
energy_used_10 = np.trapz(np.multiply(combined_datafile_single['Actual RPM'][:120],combined_datafile_single['Torque'][:120]) * (2*np.pi/60))
energy_used_per_weight = energy_used_10 / (np.mean(combined_datafile_single['Mass flow rate'][:120]) * (10 * 60) /3600)
print(energy_used_10)
energy_used_10_20 = np.trapz(np.multiply(combined_datafile_single['Actual RPM'][120:240],np.reshape(torque_predict_10_20[0,:120],(120,))) * (2*np.pi/60))
energy_used_per_weight_10_20 = energy_used_10_20 / (np.mean(combined_datafile_single['Mass flow rate'][120:320]) * (10 * 60) /3600)
print((energy_used_10_20))
energy_used_20_end = np.trapz(np.multiply(combined_datafile_single['Actual RPM'][240:len(combined_datafile_single)],np.reshape(torque_predict_10_20[0,:120],(120,))[:len(combined_datafile_single)-240]) * (2*np.pi/60))
energy_used_per_weight_20_end = energy_used_20_end / (np.mean(combined_datafile_single['Mass flow rate'][240:len(combined_datafile_single)]) * (4 * 60) /3600)
print(energy_used_20_end)
no_opt_energy = np.trapz(np.multiply(combined_datafile_single['Actual RPM'],combined_datafile_single['Torque']) * (2*np.pi/60))
energy_used_per_weight_no_opt = no_opt_energy / (np.mean(combined_datafile_single['Mass flow rate']) * (run_time * 60) /3600)
print(no_opt_energy)
print(energy_used_10+energy_used_10_20+energy_used_20_end)
print((no_opt_energy-energy_used_10-energy_used_10_20-energy_used_20_end)/no_opt_energy)

In [ ]:
torque_col = np.concatenate([np.reshape(torque_predict_10_20,(200,)),np.reshape(torque_predict_10_20,(200,))[:160]])
print(len(combined_datafile_single))
energy_full = np.trapz(np.multiply(combined_datafile_single['Actual RPM'],torque_col[:len(combined_datafile_single)])) * (2*np.pi/60)
print(energy_full)

In [ ]:
#keeping feedrate constant
min_ss_data = np.argmin(opt_data[(opt_data['FlowRate (kg/hr)']==run_feedrate) & (opt_data['final d50']>415)]['10 min energy'])
print(min_ss_data)
print(opt_data['FlowRate (kg/hr)'][min_ss_data])
print(opt_data['RPM'][min_ss_data])
print(opt_data['L/S Ratio'][min_ss_data])
print(opt_data['final d50'][min_ss_data])
combined_datafile_single['Mass flow rate'][120:len(combined_datafile_single)] = np.linspace(opt_data['FlowRate (kg/hr)'][min_ss_data],opt_data['FlowRate (kg/hr)'][min_ss_data],len(combined_datafile_single)-120)
combined_datafile_single['Actual RPM'][120:len(combined_datafile_single)] = np.linspace(opt_data['RPM'][min_ss_data],opt_data['RPM'][min_ss_data],len(combined_datafile_single)-120)
combined_datafile_single['Liquid flow rate'][120:len(combined_datafile_single)] = np.linspace(opt_data['FlowRate (kg/hr)'][min_ss_data],opt_data['FlowRate (kg/hr)'][min_ss_data],len(combined_datafile_single)-120) * opt_data['L/S Ratio'][min_ss_data]
combined_datafile_single[' D_v50'][120:len(combined_datafile_single)] = np.linspace(opt_data['final d50'][min_ss_data],opt_data['final d50'][min_ss_data],len(combined_datafile_single)-120)
# print(combined_datafile_single[' D_v50'][120:320])
print(combined_datafile_single)
lstm_input_predict_10_20 = X_scaler.fit_transform(combined_datafile_single[['Mass flow rate','7 NetWeight','Liquid flow rate','Actual RPM','Torque',' D_v50']])
lstm_input_predict_10_200 = lstm_input_predict_10_20[len(combined_datafile_single)-200:len(combined_datafile_single)]
lstm_input_predict_10_200 = lstm_input_predict_10_200.reshape(1,200,6)
torque_predict_10_20 = lstm_torque.predict(lstm_input_predict_10_200)
torque_predict_10_20 = Y_scaler_tor.inverse_transform(torque_predict_10_20)
d50_predict_10_20 = lstm_d50.predict(lstm_input_predict_10_200)
d50_predict_10_20 = Y_scaler_d50.inverse_transform(d50_predict_10_20)

In [ ]:
plt.figure()
time = np.linspace(0,600,120)
plt.plot(time,np.reshape(d50_pred_10[0,:120],(120,)))
time1 = np.linspace(600,1200,120)
plt.plot(time1,np.reshape(d50_predict_10_20[0,:120],(120,)))
time2 = np.linspace(1200,1740,108)
plt.plot(time2,np.reshape(d50_predict_10_20[0,:120],(120,))[:108])
plt.ylim([800,1000])

plt.plot(time,np.reshape(d50_pred_10[0,:120],(120,))/2.1)
plt.plot(time1,np.reshape(d50_predict_10_20[0,:120],(120,))/2.1)
plt.plot(time2,np.reshape(d50_predict_10_20[0,:120],(120,))[:108]/2.1)
plt.ylim([400,1200])
plt.legend(['D_50 exp Wet','Predicted D_50 10-20 mins Wet','Predicted D_50 20-29 mins Wet','D_50 exp Dry','Predicted D_50 10-20 mins Dry','Predicted D_50 20-29 mins Dry'])
plt.ylabel('D_50 ($\mu$m)')
plt.xlabel('Time (seconds)')
plt.figure()
plt.plot(time,np.reshape(torque_pred_10[0,:120],(120,)))
plt.plot(time1,np.reshape(torque_predict_10_20[0,:120],(120,)))
plt.plot(time2,np.reshape(torque_predict_10_20[0,:120],(120,))[:108])
plt.legend(['Torque exp','Predicted Torque 10-20 mins','Predicted Torque 20-29 mins'])
plt.xlabel('Time (seconds)')
plt.ylabel('Torque (N/m.s)')
plt.ylim([1,4])
energy_used_10 = np.trapz(np.multiply(combined_datafile_single['Actual RPM'][:120],combined_datafile_single['Torque'][:120]) * (2*np.pi/60))
energy_used_per_weight = energy_used_10 / (np.mean(combined_datafile_single['Mass flow rate'][:120]) * (10 * 60) /3600)
print(energy_used_10)
energy_used_10_20 = np.trapz(np.multiply(combined_datafile_single['Actual RPM'][120:240],np.reshape(torque_predict_10_20[0,:120],(120,))) * (2*np.pi/60))
energy_used_per_weight_10_20 = energy_used_10_20 / (np.mean(combined_datafile_single['Mass flow rate'][120:320]) * (10 * 60) /3600)
print((energy_used_10_20))
energy_used_20_end = np.trapz(np.multiply(combined_datafile_single['Actual RPM'][240:len(combined_datafile_single)],np.reshape(torque_predict_10_20[0,:120],(120,))[:len(combined_datafile_single)-240]) * (2*np.pi/60))
energy_used_per_weight_20_end = energy_used_20_end / (np.mean(combined_datafile_single['Mass flow rate'][240:len(combined_datafile_single)]) * (4 * 60) /3600)
print(energy_used_20_end)
no_opt_energy = np.trapz(np.multiply(combined_datafile_single['Actual RPM'],combined_datafile_single['Torque']) * (2*np.pi/60))
energy_used_per_weight_no_opt = no_opt_energy / (np.mean(combined_datafile_single['Mass flow rate']) * (run_time * 60) /3600)
print(no_opt_energy)
print(energy_used_10+energy_used_10_20+energy_used_20_end)
print((no_opt_energy-energy_used_10-energy_used_10_20-energy_used_20_end)/no_opt_energy)